In [1]:
# !pip install --upgrade transformers accelerate datasets sentencepiece rouge-score


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 16.9 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=ead06abab7856664b9523fc9b0eafb8478d3bce110f05b859fcb788831bfb136
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.2
    Uninstalling transformers-4.57.2:
      Successfully uninstalled transformers-4.57.2
  At

In [2]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

path = "/content/drive/MyDrive/tldr.jsonl"

Mounted at /content/drive


In [3]:
import pandas as pd

chunks = pd.read_json(path, lines=True, chunksize=20000)

sample_parts = []
count = 0
for chunk in chunks:
  sample_parts.append(chunk.sample(min(5000, len(chunk))))
  count += len(chunk)
  if count > 200000:
    break

df = pd.concat(sample_parts, ignore_index=True)
print("Shape of df: ", df.shape)
df.head()

Shape of df:  (55000, 11)


,author,body,content,content_len,id,normalizedBody,subreddit,subreddit_id,summary,summary_len,title
0,I-like-that-color,"\nShe was my best friends girl, so I knew it c...","She was my best friends girl, so I knew it cou...",190,c7s6f8k,"She was my best friends girl, so I knew it cou...",AskReddit,t5_2qh1i,Fell in love with best friends girl. They brok...,18,NaN
1,freshpiece,I work at a local grocery store. A customer wo...,I work at a local grocery store. A customer wo...,243,c4at3rf,I work at a local grocery store. A customer wo...,AskReddit,t5_2qh1i,"Man kept coming into grocery store, soliciting...",17,NaN
2,cloistered_around,"Yeah, that's just an outright lie on Nmom's pa...","Yeah, that's just an outright lie on Nmom's pa...",102,crpjtuw,"Yeah, that's just an outright lie on Nmom's pa...",raisedbynarcissists,t5_2we9n,what a liar!,3,NaN
3,afrodoc,Not necessarily the longest but definitely the...,Not necessarily the longest but definitely the...,333,cpdgg3f,Not necessarily the longest but definitely the...,AskReddit,t5_2qh1i,"Miss 9/11, shower back to sanity.",6,NaN
4,inVINC31ble,"I think that like King Arthur, there was a man...","I think that like King Arthur, there was a man...",199,ch2dxne,"I think that like King Arthur, there was a man...",TrueAtheism,t5_2soy6,"Jesus was probably real, but much more basic t...",32,NaN


In [4]:
print("Columns: ", df.columns.tolist())
print("Example dtypes: ", df.dtypes)

Columns:  ['author', 'body', 'content', 'content_len', 'id', 'normalizedBody', 'subreddit', 'subreddit_id', 'summary', 'summary_len', 'title']
Example dtypes:  author            object
body              object
content           object
content_len        int64
id                object
normalizedBody    object
subreddit         object
subreddit_id      object
summary           object
summary_len        int64
title             object
dtype: object


In [5]:
print(df.isnull().sum())

author                0
body                  0
content               0
content_len           0
id                    0
normalizedBody        0
subreddit             0
subreddit_id          0
summary               0
summary_len           0
title             52274
dtype: int64


In [6]:
display(df[['content','summary','body']].head(8))

,content,summary,body
0,"She was my best friends girl, so I knew it cou...",Fell in love with best friends girl. They brok...,"\nShe was my best friends girl, so I knew it c..."
1,I work at a local grocery store. A customer wo...,"Man kept coming into grocery store, soliciting...",I work at a local grocery store. A customer wo...
2,"Yeah, that's just an outright lie on Nmom's pa...",what a liar!,"Yeah, that's just an outright lie on Nmom's pa..."
3,Not necessarily the longest but definitely the...,"Miss 9/11, shower back to sanity.",Not necessarily the longest but definitely the...
4,"I think that like King Arthur, there was a man...","Jesus was probably real, but much more basic t...","I think that like King Arthur, there was a man..."
5,First tattoo was the day I turned 18. I got my...,"I've got a lot of ink, most of it meaningful b...",First tattoo was the day I turned 18. I got my...
6,Have a goal. Be Selfish. Just don't be a dick....,Player: Have a goal. Character: Be selfish. Bo...,Have a goal. Be Selfish. Just don't be a dick....
7,"My dad was a speed junkie for much of my life,...",My earliest memory is watching my father carom...,"My dad was a speed junkie for much of my life,..."


In [7]:
df_small = df.sample(10000, random_state=42)
X = df_small['content']
y = df_small['summary']

In [8]:
print("X samples: ", len(X))
print("Y samples: ", len(y))

X samples:  10000
Y samples:  10000


In [9]:
# checking sample input and output
print("INPUT TEXT: ", X.iloc[1])
print("OUTPUT SUMMARY: ", y.iloc[1])

INPUT TEXT:  I'm pretty sure he can't play for Germany, as he's only lived there for one year, and his parents are not German. He can play for France though (as well as Denmark, of course), since his mother is half French. But he's played for both the Danish U17s and U19, he is also born in Copenhagen, and he is 3/4 Danish, so i think he will almost surely play for the Danish national team. In fact he already said so in a Danish interview: 
 Translation:
"I've represented Denmark my entire life, both at club and at national level. It's the Danish coaches that have made me the player i am, and i don't want to piss them off (he says that), by suddenly changing national team. I'm proud of my French blood, but if Morten Olsen (Danish NT coach) calls, i will happily accept. Denmark is 100% my first priority, and i think it's foolish to think differently.
OUTPUT SUMMARY:  He CAN'T play for Germany, CAN play France, WILL play for Denmark.


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [11]:
print("Training samples: ", len(X_train))
print("Test samples: ", len(X_test))

Training samples:  8000
Test samples:  2000


In [12]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("t5-small")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [13]:
# Tokenization

max_input_len = 512
max_output_len = 128

def tokenize(batch):
  inputs = tokenizer(
      batch["content"],
      padding="max_length",
      truncation=True,
      max_length=max_input_len
  )

  outputs = tokenizer(
      batch["summary"],
      padding="max_length",
      truncation=True,
      max_length=max_output_len
  )

  inputs["labels"] = outputs["input_ids"]
  return inputs


In [14]:
from datasets import Dataset, DatasetDict
train_dataset = Dataset.from_dict({"content": X_train, "summary": y_train})
test_dataset = Dataset.from_dict({"content": X_test, "summary": y_test})

dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

small_dataset = dataset['train'].select(range(5000))
test_dataset = dataset['test'].select(range(1000))

dataset = DatasetDict({
    "train": small_dataset,
    "test": test_dataset
})



In [15]:
tokenized_dataset = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

## Training 5 models

In [16]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer
import torch

def train_model(model_name, tokenized_dataset):
  print(f"\nTraining model {model_name}\n")

  # loading tokenizer and model
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

  data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

  # trainings args
  args = TrainingArguments(
    output_dir=f"./results/{model_name}",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    learning_rate=2e-4,
    eval_strategy="epoch",
    save_strategy="no",
    logging_steps=20,
    fp16=True
)

   # Trainer
  trainer = Trainer(
        model=model,
        tokenizer=tokenizer,
        args=args,
        data_collator=data_collator,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["test"],
    )

  # Train
  trainer.train()

  # Save
  model.save_pretrained(f"./saved_models/{model_name}")
  tokenizer.save_pretrained(f"./saved_models/{model_name}")

  print(f"Done training {model_name}")


In [17]:
models = [
    "t5-small",
    "t5-base",
    "facebook/bart-base",
    "facebook/bart-large-cnn",
    "google/pegasus-xsum",
]

In [18]:
for m in models:
  try:
    train_model(m, tokenized_dataset)
  except Exception as e:
    print(f"Failed on {m}: {e}")


Training model t5-small



config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/tmp/ipython-input-2349006243.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

...

generation_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

/tmp/ipython-input-2349006243.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.


Epoch,Training Loss,Validation Loss
1,1.521300,1.712878


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:3918: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 64, 'num_beams': 8, 'length_penalty': 0.6}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Done training google/pegasus-xsum


## Training Best Models (t5-small and t5-base)

In [ ]:
def train_model(model_name, tokenized_dataset):
  print(f"\nTraining model {model_name}\n")

  # loading tokenizer and model
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

  data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

  # trainings args

  args = TrainingArguments(
    output_dir="./t5-small-output",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=200,
    learning_rate=5e-5,
    weight_decay=0.01,
    fp16=True,
)

   # Trainer
  trainer = Trainer(
        model=model,
        tokenizer=tokenizer,
        args=args,
        data_collator=data_collator,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["test"],
    )

  # Train
  trainer.train()

  # Save
  model.save_pretrained(f"./saved/{model_name}")
  tokenizer.save_pretrained(f"./saved/{model_name}")

  print(f"Done training {model_name}")



In [ ]:
best_model = "t5-small"
try:
  train_model(best_model, tokenized_dataset)
except Exception as e:
  print(f"Failed on {best_model}: {e}")


Training model t5-small



/tmp/ipython-input-1804522054.py:39: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sp23-bai-014 (sp23-bai-014-comsats-university-islamabad) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss
1,0.953100,0.915544


Done training t5-small


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_path = "./saved/t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [ ]:
text = """
Well, unless you were a legitimate group of people claiming self-determination as a nation-state,
it would be near impossible. Without an over-arching regulatory body it is necessary to get
recognition from a majority of world states (or at least powerful states, see UNSC permanent members).
Look at a country like Kosovo which has been recognised as a legitimate state by the U.S. and the U.K.
and (I believe) China, but not Russia (which lends its position to Serbian influence). It can still
operate as a state with regard to some of its trade, diplomatic, military relations with other states
however Serbia would not perceive it as such when it deals with Kosovo
"""

inputs = tokenizer(
    text,
    return_tensors="pt",
    max_length=512,
    truncation=True
)

output_ids = model.generate(
    inputs["input_ids"],
    max_length=100,
    num_beams=4,
    early_stopping=True
)

print("SUMMARY:\n", tokenizer.decode(output_ids[0], skip_special_tokens=True))


SUMMARY:
 Unless you were a legitimate group of people claiming self-determination as a nation-state, it would be near impossible.


In [ ]:
#!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.6 MB/s eta 0:00:00


In [ ]:
import evaluate

rouge = evaluate.load("rouge")

def generate_summary(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    output = model.generate(inputs["input_ids"], max_length=100, num_beams=4)
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
predictions = []
references = []

for sample in tokenized_dataset["test"]:
    pred = generate_summary(sample["content"])
    predictions.append(pred)
    references.append(sample["summary"])

In [ ]:
results = rouge.compute(
    predictions=predictions,
    references=references
)

print(results)


{'rouge1': np.float64(0.13621358591809515), 'rouge2': np.float64(0.02856247012104633), 'rougeL': np.float64(0.11196517897858314), 'rougeLsum': np.float64(0.11271585392803547)}


In [ ]:
best_model = "t5-base"
try:
  train_model(best_model, tokenized_dataset)
except Exception as e:
  print(f"Failed on {best_model}: {e}")


Training model t5-base



config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

...

/tmp/ipython-input-3608443347.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sp23-bai-014 (sp23-bai-014-comsats-university-islamabad) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.903600,0.792482


Done training t5-base


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_path = "./saved/t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

def generate_summary(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True)
    output = model.generate(inputs["input_ids"], max_length=100, num_beams=4)
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
predictions = []
references = []

for sample in tokenized_dataset["test"]:
    pred = generate_summary(sample["content"])
    predictions.append(pred)
    references.append(sample["summary"])

In [ ]:
results = rouge.compute(
    predictions=predictions,
    references=references
)

print(results)


{'rouge1': np.float64(0.1498890841108344), 'rouge2': np.float64(0.03691607572419636), 'rougeL': np.float64(0.12462733269067586), 'rougeLsum': np.float64(0.12608628289318202)}
